Introduction to Agents with LangGraph
Simple LLM calls are limited by the knowledge the model was trained on. To overcome this, developers use Chains or Agentic Systems to give LLMs more control over application workflows.

Chains vs. Agents
Chains – Define fixed steps before and after an LLM call, ensuring reliable execution.

Agents – Allow the LLM to decide the flow of execution dynamically.

Why LangGraph?
Giving more control to an LLM reduces reliability—but LangGraph helps maintain reliability while enabling agentic behavior.

More flexibility in designing workflows.
Clear visibility into control flow (no hidden complex prompts).
Supports cycles and persistence, making it suitable for iterative AI workflows.
Graphs in LangGraph
LangGraph structures workflows as graphs with nodes and edges:

Nodes – Python functions that define agent logic.

Edges – Python functions that decide which node runs next based on state.

Graph-Based AI Workflows
Graphs are widely used in AI and data-driven applications:

Social Networks – Users (nodes) connected by relationships (edges).
E-Commerce – Items and customers as nodes, with purchasing behavior forming edges.
In LangGraph, this concept is applied to AI workflows, where decision-making paths are dynamically determined.

Integration with LangChain
LangGraph works independently, but it integrates seamlessly with LangChain, allowing developers to:

Use LangChain components to build nodes and edges.
Leverage LangGraph for workflow orchestration.
Final Thoughts
LangGraph enables structured, agent-driven AI applications while keeping control and reliability in the hands of developers. By combining LangChain and LangGraph, it’s possible to build dynamic, intelligent applications that adapt to user inputs and external data.

Agentic Workflows with LangGraph
Agentic workflows allow LLMs to control execution flow, adapting to user input, conditions, and tools. Unlike fixed Chains, agentic workflows make real-time decisions, improving flexibility and automation.

Setting Up the Workflow
Instantiate the Model – Define an LLM and optionally bind tools to expand its capabilities.
llm_with_tools = llm.bind_tools([tool_a, tool_b, tool_c], tool_choice="auto")
Initialize the Graph – Create a StateGraph to track input, output, and intermediate data.
workflow = StateGraph(MessagesState)
Define Nodes – Each node represents a step in the workflow, processing input and modifying the state.
def first_node(state):
    return {"results": f"Hello, {state['input']}!"}
workflow.add_node(first_node)
Create Edges – Define execution paths between nodes.
workflow.add_edge("node_a", "node_b")
Specify Start and End – Ensure the workflow has a defined entry and exit.
workflow.add_edge(START, "node_a")
workflow.add_edge("node_b", END)
Compile the Workflow – Validate structure before execution.
graph = workflow.compile()
Executing and Visualizing
Invoke the workflow with input and configuration.

graph.invoke({"input": "Some input"})
Use Mermaid diagrams to visualize the process for debugging and documentation.

Final Thoughts
LangGraph enables dynamic, AI-driven workflows by combining state tracking, flexible decision-making, and structured execution. By integrating nodes, edges, and visualization, developers can create scalable, intelligent automation.

## L3_demo_01_langgraph_workflow

1. Basic Data Processing Workflow  
a. State Definition  
A State class is created by extending TypedDict.
State holds:
input: integer
output: integer
class State(TypedDict):
  input: int
  output: int   
b. Node A  
Processes the input value.
Adds a random offset (1–10) to the input.
Updates the output field.
def node_a(state):
  offset = random.randint(1, 10)
  output = state["input"] + offset
  return {"output": output}  
c. Node B  
Takes the output of Node A as its input.
Adds another random offset.
Updates the output field again.
def node_b(state):  
  offset = random.randint(1, 10)
  output = state["output"] + offset
  return {"output": output}  
d. Workflow Construction  
A StateGraph is created.
Nodes are added.
Edges are set:
start → node_a → node_b → end
workflow = StateGraph(State)
workflow.add_node("node_a", node_a)
workflow.add_node("node_b", node_b)
workflow.add_edge("start", "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", "end")  
e. Execution Example  
Input: 1
Example run:
Node A: 1 + offset 1 → output 2
Node B: 2 + offset 6 → output 8
Each run may vary due to random offsets.


In [ ]:
import random
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from IPython.display import Image, display

2. LLM-Based Workflow  
a. State Definition  
A second State class is created for the LLM use case:
question: string
response: string
class State(TypedDict):
  question: str
  response: str  
b. LLM Node  
A node uses a chat LLM to answer Pokémon-related questions.
Composes messages:  
SystemMessage: "You are a Pokémon specialist."
HumanMessage: User-provided question.
The model generates a response which is stored in the state.
def model_node(state):  
  messages = [
      SystemMessage(content="You are a Pokémon specialist."),
      HumanMessage(content=state["question"])
  ]
  response = llm.invoke(messages)
  return {"response": response.content}


In [ ]:
class State(TypedDict):
    input: int
    output: int

In [ ]:
def node_a(state: State)->State:
    input_value = state['input']
    offset = random.randint(1,10)
    output =  input_value + offset
    print(
        f"NODE A:\n "
        f"->input:{input_value}\n " 
        f"->offset:{offset}\n "
        f"->output:{output}\n "
    )
    return State(output=output)


In [ ]:
def node_b(state: State):
    input_value = state['output']
    offset = random.randint(1,10)
    output =  input_value + offset
    print(
        f"NODE B:\n "
        f"->input:{input_value}\n " 
        f"->offset:{offset}\n "
        f"->output:{output}\n "
    )
    return {"output": output}

c. Workflow Construction  
Only one node (model) is added.
Edges:
start → model → end
workflow = StateGraph(State)
workflow.add_node("model", model_node)
workflow.add_edge("start", "model")
workflow.add_edge("model", "end")  
d. Execution Example  
Question: "What is the name of Ash's first Pokémon?"
Correct output: "Ash's first Pokémon is Pikachu."

In [ ]:
workflow = StateGraph(state_schema=State)

In [ ]:
workflow.add_node(node_a)
workflow.add_node(node_b)

In [ ]:
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

In [ ]:
graph = workflow.compile()

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
graph.invoke(
    input = {
        "input": 1,
    }, 
)

Call LLMs

In [ ]:
# or ust use the preset
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [ ]:
class State(TypedDict):
    question:str
    response:str

In [ ]:
def model(state: State):
    question = state["question"]
    response = llm.invoke([
        SystemMessage("You're a Pokémon specialist"),
        HumanMessage(question)
    ])

    return {"response": response.content}

In [ ]:
workflow = StateGraph(State)

In [ ]:
workflow.add_node("model", model)

In [ ]:
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

In [ ]:
graph = workflow.compile()

In [ ]:
display(
    Image(
        graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
result = graph.invoke(
    input={
        "question": "What's the name of Ash's first pokémon?"
    }, 
)

In [ ]:
result

3. Key Concepts Highlighted  
Sequential flow: Data passes from one node to the next.
TypedDict states: Enforce predictable data structure across nodes.
Node outputs feed node inputs: Node A’s output becomes Node B’s input.
Graphs vs. Chains: Unlike LangChain chains, LangGraph explicitly defines nodes and edges for modular flow control.
Random vs. Deterministic processing: Shows use of randomness in basic workflows and structured, reproducible behavior in LLM flows.  
4. Conclusion  
LangGraph enables building modular, sequential workflows for both simple data tasks and complex AI-driven applications.
It sets a strong foundation for more advanced multi-node and multi-agent workflows.